In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd

In [ ]:
# spotify object to access API
client_credential_manager = SpotifyClientCredentials(client_id='client id',
                                    client_secret='client secret')
sp = spotipy.Spotify(client_credentials_manager=client_credential_manager)

## Demographic Spotify Song by Country

### Indonesian Popular Track Song

In [ ]:
# buat list kosong untuk menampung hasil penarikan lagu dari track
artist_name1 = []
track_name1 = []
popularity1 = []
track_id1 = []

# for in buat fetch data
for i in range(0, 1000, 50):
    # maksimum offset cuma sampe 1000
    track_results1 = sp.search(q='year:2022', type='track',market='ID', limit=50, offset=i)
    for i,t in enumerate(track_results1['tracks']['items']):
        artist_name1.append(t['artists'][0]['name'])
        track_name1.append(t['name'])
        track_id1.append(t['id'])
        popularity1.append(t['popularity'])

# checking the result
print(f'number of elements in the track_id list : {len(track_id1)}')

number of elements in the track_id list : 1000


In [ ]:
# tampung semua data hasil tarikan ke dataframe
indo_track = pd.DataFrame({'artist name' : artist_name1, 'track name' : track_name1, 
                          'track id' : track_id1, 'popularity' : popularity1})
indo_track.head()

,artist name,track name,track id,popularity
0,Keisya Levronka,Tak Ingin Usai,1b0aDgrG775i4sPQgJGmkU,84
1,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,22
2,Joji,Glimpse of Us,6xGruZOHLs39ZbVccQTuPZ,99
3,Lana Del Rey,Summertime Sadness (Lana Del Rey Vs. Cedric Ge...,1kA8kyl733T0Qmy8rOIYeM,3
4,Tulus,Hati-Hati di Jalan,2hHeGD57S0BcopfVcmehdl,82


In [ ]:
# liat jumlah baris dan kolom dari dataframe track
indo_track.shape

(1000, 4)

In [ ]:
# liat informasi dataset track
indo_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist name  1000 non-null   object
 1   track name   1000 non-null   object
 2   track id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
# groupby berdasarkan nama artist dan nama track untuk memeriksa duplicate
grouped_indo = indo_track.groupby(['artist name', 'track name'], as_index=True).size()
grouped_indo[grouped_indo > 1].count()

94

In [ ]:
# menghapus duplicate pada nama artist dan nama tracknya
indo_track.drop_duplicates(subset=['artist name', 'track name'], inplace=True)

In [ ]:
# groupby berdasarkan nama artist dan nama track buat memeriksa ulang duplicate
group_id_after_dropping = indo_track.groupby(['artist name', 'track name'], as_index=True).size()
group_id_after_dropping[group_id_after_dropping > 1].count()

0

In [ ]:
# check duplicate ulang per kolom berdasasrkan nama artist dan nama track
indo_track[indo_track.duplicated(subset=['artist name', 'track name'], keep=False)].count()

artist name    0
track name     0
track id       0
popularity     0
dtype: int64

In [ ]:
# liat jumlah baris dan kolom setelah penghapusan duplicate
indo_track.shape

(754, 4)

Ekstrack Audio Feature nya

In [ ]:
# list kosong, 100 batchsize dan counter buat nampung none result
row_id = []
batch_size = 100
# none result berarti yang tidak ada audio features
none_counter = 0

for i in range(0, len(indo_track['track id']), batch_size):
    batch = indo_track['track id'][i:i+batch_size]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        # kalau track yang ditarik bukan audio feature
        if t == None:
            none_counter = none_counter + 1
        # kalau track yang ditarik adalah audio feature
        else:
            row_id.append(t)

# liat jumlah track yang bukan fitur audio
print(f'Jumlah track yang bukan Audio Features yang ada : {none_counter}')
# liat hasil dari jumlah elemen pada track 
print(f'jumlah elemen yang ada di dalam list track_id : {len(row_id)}')

Jumlah track yang bukan Audio Features yang ada : 2
jumlah elemen yang ada di dalam list track_id : 752


In [ ]:
# masukan semua elemen dalam list track ke dalam dataframe
indo_aufeat = pd.DataFrame.from_dict(row_id, orient='columns')
indo_aufeat.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.490,0.398,0,-7.712,1,0.0266,0.8150,0.000187,0.114,0.136,129.772,audio_features,1b0aDgrG775i4sPQgJGmkU,spotify:track:1b0aDgrG775i4sPQgJGmkU,https://api.spotify.com/v1/tracks/1b0aDgrG775i...,https://api.spotify.com/v1/audio-analysis/1b0a...,278500,3
1,0.521,0.683,5,-6.240,0,0.5010,0.1590,0.000000,0.187,0.737,139.439,audio_features,4eHkG6yWQ4pD0zqVkV0R5a,spotify:track:4eHkG6yWQ4pD0zqVkV0R5a,https://api.spotify.com/v1/tracks/4eHkG6yWQ4pD...,https://api.spotify.com/v1/audio-analysis/4eHk...,161808,3
2,0.440,0.317,8,-9.258,1,0.0531,0.8910,0.000005,0.141,0.268,169.914,audio_features,6xGruZOHLs39ZbVccQTuPZ,spotify:track:6xGruZOHLs39ZbVccQTuPZ,https://api.spotify.com/v1/tracks/6xGruZOHLs39...,https://api.spotify.com/v1/audio-analysis/6xGr...,233456,3
3,0.572,0.810,1,-5.791,0,0.0557,0.0147,0.000007,0.130,0.106,126.045,audio_features,1kA8kyl733T0Qmy8rOIYeM,spotify:track:1kA8kyl733T0Qmy8rOIYeM,https://api.spotify.com/v1/tracks/1kA8kyl733T0...,https://api.spotify.com/v1/audio-analysis/1kA8...,214912,4
4,0.643,0.436,6,-11.322,1,0.0357,0.7020,0.088900,0.115,0.755,71.969,audio_features,2hHeGD57S0BcopfVcmehdl,spotify:track:2hHeGD57S0BcopfVcmehdl,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,https://api.spotify.com/v1/audio-analysis/2hHe...,242000,4


In [ ]:
# liat ukuram jumlah baris dan kolom dari dataframe audio features
indo_aufeat.shape

(752, 18)

In [ ]:
# liat info dari dataframe audio feature
indo_aufeat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      752 non-null    float64
 1   energy            752 non-null    float64
 2   key               752 non-null    int64  
 3   loudness          752 non-null    float64
 4   mode              752 non-null    int64  
 5   speechiness       752 non-null    float64
 6   acousticness      752 non-null    float64
 7   instrumentalness  752 non-null    float64
 8   liveness          752 non-null    float64
 9   valence           752 non-null    float64
 10  tempo             752 non-null    float64
 11  type              752 non-null    object 
 12  id                752 non-null    object 
 13  uri               752 non-null    object 
 14  track_href        752 non-null    object 
 15  analysis_url      752 non-null    object 
 16  duration_ms       752 non-null    int64  
 1

In [ ]:
# menghapus kolom yang tidak digunakan
columns_to_drop = ['analysis_url', 'track_href', 'type', 'uri']

indo_aufeat.drop(columns_to_drop, axis=1, inplace=True)
# mengganti kolom id jadi id track
indo_aufeat.rename(columns={'id': 'track id'}, inplace=True)
indo_aufeat.shape

(752, 14)

In [ ]:
# merge dataframe, antara df track dan fitur audio
# metode 'inner' memastikan kalau cuma keep track id yang sekarang di kedua dataset
indo_song_df = pd.merge(indo_track, indo_aufeat, on='track id', how='inner')
indo_song_df.head()

,artist name,track name,track id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Keisya Levronka,Tak Ingin Usai,1b0aDgrG775i4sPQgJGmkU,84,0.490,0.398,0,-7.712,1,0.0266,0.8150,0.000187,0.114,0.136,129.772,278500,3
1,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,22,0.521,0.683,5,-6.240,0,0.5010,0.1590,0.000000,0.187,0.737,139.439,161808,3
2,Joji,Glimpse of Us,6xGruZOHLs39ZbVccQTuPZ,99,0.440,0.317,8,-9.258,1,0.0531,0.8910,0.000005,0.141,0.268,169.914,233456,3
3,Lana Del Rey,Summertime Sadness (Lana Del Rey Vs. Cedric Ge...,1kA8kyl733T0Qmy8rOIYeM,3,0.572,0.810,1,-5.791,0,0.0557,0.0147,0.000007,0.130,0.106,126.045,214912,4
4,Tulus,Hati-Hati di Jalan,2hHeGD57S0BcopfVcmehdl,82,0.643,0.436,6,-11.322,1,0.0357,0.7020,0.088900,0.115,0.755,71.969,242000,4


In [ ]:
# liat ukuran jumlah baris dan kolom dataframe audio features
indo_aufeat.shape

(752, 14)

In [ ]:
# liat info dataframe gabungan 
indo_song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 751
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist name       752 non-null    object 
 1   track name        752 non-null    object 
 2   track id          752 non-null    object 
 3   popularity        752 non-null    int64  
 4   danceability      752 non-null    float64
 5   energy            752 non-null    float64
 6   key               752 non-null    int64  
 7   loudness          752 non-null    float64
 8   mode              752 non-null    int64  
 9   speechiness       752 non-null    float64
 10  acousticness      752 non-null    float64
 11  instrumentalness  752 non-null    float64
 12  liveness          752 non-null    float64
 13  valence           752 non-null    float64
 14  tempo             752 non-null    float64
 15  duration_ms       752 non-null    int64  
 16  time_signature    752 non-null    int64  
dt

In [ ]:
# periksa jumlah duplicate pada kolom artist name dan track name
indo_song_df[indo_song_df.duplicated(subset=['artist name', 'track name'], keep=False)].count()

artist name         0
track name          0
track id            0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64

### Korean Popular Track Song

In [ ]:
# buat list kosong untuk menampung hasil penarikan lagu dari track
artist_name2 = []
track_name2 = []
popularity2 = []
track_id2 = []

# for in buat fetch data
for i in range(0, 1000, 50):
    # maksimum offset cuma sampe 1000
    track_results2 = sp.search(q='year:2022', type='track',market='KR', limit=50, offset=i)
    for i,t in enumerate(track_results2['tracks']['items']):
        artist_name2.append(t['artists'][0]['name'])
        track_name2.append(t['name'])
        track_id2.append(t['id'])
        popularity2.append(t['popularity'])

# checking the result
print(f'number of elements in the track_id list : {len(track_id2)}')

number of elements in the track_id list : 1000


In [ ]:
# tampung semua data hasil tarikan ke dataframe
korean_track = pd.DataFrame({'artist name' : artist_name2, 'track name' : track_name2, 
                          'track id' : track_id2, 'popularity' : popularity2})
korean_track.head()

,artist name,track name,track id,popularity
0,Jimin,With you,2gzhQaCTeNgxpeB2TPllyY,86
1,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,22
2,Charlie Puth,Left and Right (Feat. Jung Kook of BTS),0mBP9X2gPCuapvpZ7TGDk3,94
3,Lana Del Rey,Summertime Sadness (Lana Del Rey Vs. Cedric Ge...,1kA8kyl733T0Qmy8rOIYeM,3
4,BTS,Yet To Come,10SRMwb9EuVS1K9rYsBfHQ,93


In [ ]:
# liat jumlah baris dan kolom dari dataframe track
korean_track.shape

(1000, 4)

In [ ]:
# liat informasi dataset track
korean_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist name  1000 non-null   object
 1   track name   1000 non-null   object
 2   track id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
# groupby berdasarkan nama artist dan nama track untuk memeriksa duplicate
grouped_korean = korean_track.groupby(['artist name', 'track name'], as_index=True).size()
grouped_korean[grouped_korean > 1].count()

88

In [ ]:
# menghapus duplicate pada nama artist dan nama tracknya
korean_track.drop_duplicates(subset=['artist name', 'track name'], inplace=True)

In [ ]:
# groupby berdasarkan nama artist dan nama track buat memeriksa ulang duplicate
group_kr_after_dropping = korean_track.groupby(['artist name', 'track name'], as_index=True).size()
group_kr_after_dropping[group_kr_after_dropping > 1].count()

0

In [ ]:
# check duplicate ulang per kolom berdasasrkan nama artist dan nama track
korean_track[korean_track.duplicated(subset=['artist name', 'track name'], keep=False)].count()

artist name    0
track name     0
track id       0
popularity     0
dtype: int64

In [ ]:
# liat jumlah baris dan kolom setelah penghapusan duplicate
korean_track.shape

(789, 4)

Ekstrack Audio Feature nya

In [ ]:
# list kosong, 100 batchsize dan counter buat nampung none result
row_kr = []
batch_size = 100
# none result berarti yang tidak ada audio features
none_counter = 0

for i in range(0, len(korean_track['track id']), batch_size):
    batch = korean_track['track id'][i:i+batch_size]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        # kalau track yang ditarik bukan audio feature
        if t == None:
            none_counter = none_counter + 1
        # kalau track yang ditarik adalah audio feature
        else:
            row_kr.append(t)

# liat jumlah track yang bukan fitur audio
print(f'Jumlah track yang bukan Audio Features yang ada : {none_counter}')
# liat hasil dari jumlah elemen pada track 
print(f'jumlah elemen yang ada di dalam list track_id : {len(row_kr)}')

Jumlah track yang bukan Audio Features yang ada : 1
jumlah elemen yang ada di dalam list track_id : 788


In [ ]:
# masukan semua elemen dalam list track ke dalam dataframe
korean_aufeat = pd.DataFrame.from_dict(row_kr, orient='columns')
korean_aufeat.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.532,0.426,3,-6.117,1,0.0288,0.6390,0.000000,0.1340,0.137,149.898,audio_features,2gzhQaCTeNgxpeB2TPllyY,spotify:track:2gzhQaCTeNgxpeB2TPllyY,https://api.spotify.com/v1/tracks/2gzhQaCTeNgx...,https://api.spotify.com/v1/audio-analysis/2gzh...,201905,3
1,0.521,0.683,5,-6.240,0,0.5010,0.1590,0.000000,0.1870,0.737,139.439,audio_features,4eHkG6yWQ4pD0zqVkV0R5a,spotify:track:4eHkG6yWQ4pD0zqVkV0R5a,https://api.spotify.com/v1/tracks/4eHkG6yWQ4pD...,https://api.spotify.com/v1/audio-analysis/4eHk...,161808,3
2,0.881,0.592,2,-4.898,1,0.0324,0.6190,0.000013,0.0901,0.719,101.058,audio_features,0mBP9X2gPCuapvpZ7TGDk3,spotify:track:0mBP9X2gPCuapvpZ7TGDk3,https://api.spotify.com/v1/tracks/0mBP9X2gPCua...,https://api.spotify.com/v1/audio-analysis/0mBP...,154487,4
3,0.572,0.810,1,-5.791,0,0.0557,0.0147,0.000007,0.1300,0.106,126.045,audio_features,1kA8kyl733T0Qmy8rOIYeM,spotify:track:1kA8kyl733T0Qmy8rOIYeM,https://api.spotify.com/v1/tracks/1kA8kyl733T0...,https://api.spotify.com/v1/audio-analysis/1kA8...,214912,4
4,0.559,0.885,1,-4.515,1,0.1890,0.0432,0.000000,0.3330,0.683,172.029,audio_features,10SRMwb9EuVS1K9rYsBfHQ,spotify:track:10SRMwb9EuVS1K9rYsBfHQ,https://api.spotify.com/v1/tracks/10SRMwb9EuVS...,https://api.spotify.com/v1/audio-analysis/10SR...,193594,4


In [ ]:
# liat ukuram jumlah baris dan kolom dari dataframe audio features
korean_aufeat.shape

(788, 18)

In [ ]:
# liat info dari dataframe audio feature
korean_aufeat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      788 non-null    float64
 1   energy            788 non-null    float64
 2   key               788 non-null    int64  
 3   loudness          788 non-null    float64
 4   mode              788 non-null    int64  
 5   speechiness       788 non-null    float64
 6   acousticness      788 non-null    float64
 7   instrumentalness  788 non-null    float64
 8   liveness          788 non-null    float64
 9   valence           788 non-null    float64
 10  tempo             788 non-null    float64
 11  type              788 non-null    object 
 12  id                788 non-null    object 
 13  uri               788 non-null    object 
 14  track_href        788 non-null    object 
 15  analysis_url      788 non-null    object 
 16  duration_ms       788 non-null    int64  
 1

In [ ]:
# menghapus kolom yang tidak digunakan
columns_to_drop = ['analysis_url', 'track_href', 'type', 'uri']

korean_aufeat.drop(columns_to_drop, axis=1, inplace=True)
# mengganti kolom id jadi id track
korean_aufeat.rename(columns={'id': 'track id'}, inplace=True)
korean_aufeat.shape

(788, 14)

In [ ]:
# merge dataframe, antara df track dan fitur audio
# metode 'inner' memastikan kalau cuma keep track id yang sekarang di kedua dataset
korean_song_df = pd.merge(korean_track, korean_aufeat, on='track id', how='inner')
korean_song_df.head()

,artist name,track name,track id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jimin,With you,2gzhQaCTeNgxpeB2TPllyY,86,0.532,0.426,3,-6.117,1,0.0288,0.6390,0.000000,0.1340,0.137,149.898,201905,3
1,Ish Kevin,My Year (2022),4eHkG6yWQ4pD0zqVkV0R5a,22,0.521,0.683,5,-6.240,0,0.5010,0.1590,0.000000,0.1870,0.737,139.439,161808,3
2,Charlie Puth,Left and Right (Feat. Jung Kook of BTS),0mBP9X2gPCuapvpZ7TGDk3,94,0.881,0.592,2,-4.898,1,0.0324,0.6190,0.000013,0.0901,0.719,101.058,154487,4
3,Lana Del Rey,Summertime Sadness (Lana Del Rey Vs. Cedric Ge...,1kA8kyl733T0Qmy8rOIYeM,3,0.572,0.810,1,-5.791,0,0.0557,0.0147,0.000007,0.1300,0.106,126.045,214912,4
4,BTS,Yet To Come,10SRMwb9EuVS1K9rYsBfHQ,93,0.559,0.885,1,-4.515,1,0.1890,0.0432,0.000000,0.3330,0.683,172.029,193594,4


In [ ]:
# liat ukuran jumlah baris dan kolom dataframe audio features
korean_aufeat.shape

(788, 14)

In [ ]:
# liat info dataframe gabungan 
korean_song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788 entries, 0 to 787
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist name       788 non-null    object 
 1   track name        788 non-null    object 
 2   track id          788 non-null    object 
 3   popularity        788 non-null    int64  
 4   danceability      788 non-null    float64
 5   energy            788 non-null    float64
 6   key               788 non-null    int64  
 7   loudness          788 non-null    float64
 8   mode              788 non-null    int64  
 9   speechiness       788 non-null    float64
 10  acousticness      788 non-null    float64
 11  instrumentalness  788 non-null    float64
 12  liveness          788 non-null    float64
 13  valence           788 non-null    float64
 14  tempo             788 non-null    float64
 15  duration_ms       788 non-null    int64  
 16  time_signature    788 non-null    int64  
dt

In [ ]:
# periksa jumlah duplicate pada kolom artist name dan track name
korean_song_df[korean_song_df.duplicated(subset=['artist name', 'track name'], keep=False)].count()

artist name         0
track name          0
track id            0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64

### Japan Popular Track Song

In [ ]:
# buat list kosong untuk menampung hasil penarikan lagu dari track
artist_name3 = []
track_name3 = []
popularity3 = []
track_id3 = []

# for in buat fetch data
for i in range(0, 1000, 50):
    # maksimum offset cuma sampe 1000
    track_results3 = sp.search(q='year:2022', type='track',market='JP', limit=50, offset=i)
    for i,t in enumerate(track_results3['tracks']['items']):
        artist_name3.append(t['artists'][0]['name'])
        track_name3.append(t['name'])
        track_id3.append(t['id'])
        popularity3.append(t['popularity'])

# checking the result
print(f'number of elements in the track_id list : {len(track_id3)}')

number of elements in the track_id list : 1000


In [ ]:
# tampung semua data hasil tarikan ke dataframe
japan_track = pd.DataFrame({'artist name' : artist_name3, 'track name' : track_name3, 
                          'track id' : track_id3, 'popularity' : popularity3})
japan_track.head()

,artist name,track name,track id,popularity
0,Official HIGE DANdism,ミックスナッツ,60ReNXcPcsVFd7nUuQw5HZ,77
1,Ish Kevin,My Year (2022),16TeLUubmneRAhwmRAMbjM,21
2,Kenshi Yonezu,POP SONG,2wPSiRWMFFeL1FC4MJ3Qxf,50
3,THE YELLOW MONKEY,JAM - 2022 Remaster,4uQkimBWBbDdDZdPo6GJsZ,46
4,SEKAI NO OWARI,Habit,2uMNMcjjUz8oNIxjIu20qE,78


In [ ]:
# liat jumlah baris dan kolom dari dataframe track
japan_track.shape

(1000, 4)

In [ ]:
# liat informasi dataset track
japan_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist name  1000 non-null   object
 1   track name   1000 non-null   object
 2   track id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
# groupby berdasarkan nama artist dan nama track untuk memeriksa duplicate
grouped_japan = japan_track.groupby(['artist name', 'track name'], as_index=True).size()
grouped_japan[grouped_japan > 1].count()

75

In [ ]:
# menghapus duplicate pada nama artist dan nama tracknya
japan_track.drop_duplicates(subset=['artist name', 'track name'], inplace=True)

In [ ]:
# groupby berdasarkan nama artist dan nama track buat memeriksa ulang duplicate
group_jp_after_dropping = japan_track.groupby(['artist name', 'track name'], as_index=True).size()
group_jp_after_dropping[group_jp_after_dropping > 1].count()

0

In [ ]:
# check duplicate ulang per kolom berdasasrkan nama artist dan nama track
japan_track[japan_track.duplicated(subset=['artist name', 'track name'], keep=False)].count()

artist name    0
track name     0
track id       0
popularity     0
dtype: int64

In [ ]:
# liat jumlah baris dan kolom setelah penghapusan duplicate
japan_track.shape

(834, 4)

In [ ]:
# list kosong, 100 batchsize dan counter buat nampung none result
row_jp = []
batch_size = 100
# none result berarti yang tidak ada audio features
none_counter = 0

for i in range(0, len(japan_track['track id']), batch_size):
    batch = japan_track['track id'][i:i+batch_size]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        # kalau track yang ditarik bukan audio feature
        if t == None:
            none_counter = none_counter + 1
        # kalau track yang ditarik adalah audio feature
        else:
            row_jp.append(t)

# liat jumlah track yang bukan fitur audio
print(f'Jumlah track yang bukan Audio Features yang ada : {none_counter}')
# liat hasil dari jumlah elemen pada track 
print(f'jumlah elemen yang ada di dalam list track_id : {len(row_jp)}')

Jumlah track yang bukan Audio Features yang ada : 1
jumlah elemen yang ada di dalam list track_id : 833


In [ ]:
# masukan semua elemen dalam list track ke dalam dataframe
japan_aufeat = pd.DataFrame.from_dict(row_jp, orient='columns')
japan_aufeat.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.542,0.918,6,-5.074,1,0.0937,0.01010,0.000000,0.259,0.784,150.094,audio_features,60ReNXcPcsVFd7nUuQw5HZ,spotify:track:60ReNXcPcsVFd7nUuQw5HZ,https://api.spotify.com/v1/tracks/60ReNXcPcsVF...,https://api.spotify.com/v1/audio-analysis/60Re...,213438,4
1,0.521,0.683,5,-6.240,0,0.5010,0.15900,0.000000,0.187,0.737,139.439,audio_features,16TeLUubmneRAhwmRAMbjM,spotify:track:16TeLUubmneRAhwmRAMbjM,https://api.spotify.com/v1/tracks/16TeLUubmneR...,https://api.spotify.com/v1/audio-analysis/16Te...,161808,3
2,0.762,0.792,11,-6.443,0,0.1340,0.22000,0.000017,0.939,0.806,109.968,audio_features,2wPSiRWMFFeL1FC4MJ3Qxf,spotify:track:2wPSiRWMFFeL1FC4MJ3Qxf,https://api.spotify.com/v1/tracks/2wPSiRWMFFeL...,https://api.spotify.com/v1/audio-analysis/2wPS...,199187,4
3,0.282,0.862,0,-3.578,1,0.0484,0.00392,0.000000,0.107,0.366,113.808,audio_features,4uQkimBWBbDdDZdPo6GJsZ,spotify:track:4uQkimBWBbDdDZdPo6GJsZ,https://api.spotify.com/v1/tracks/4uQkimBWBbDd...,https://api.spotify.com/v1/audio-analysis/4uQk...,319987,3
4,0.749,0.850,10,-5.390,0,0.1260,0.20300,0.000000,0.180,0.912,117.968,audio_features,2uMNMcjjUz8oNIxjIu20qE,spotify:track:2uMNMcjjUz8oNIxjIu20qE,https://api.spotify.com/v1/tracks/2uMNMcjjUz8o...,https://api.spotify.com/v1/audio-analysis/2uMN...,253154,4


In [ ]:
# liat ukuram jumlah baris dan kolom dari dataframe audio features
japan_aufeat.shape

(833, 18)

In [ ]:
# liat info dari dataframe audio feature
japan_aufeat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      833 non-null    float64
 1   energy            833 non-null    float64
 2   key               833 non-null    int64  
 3   loudness          833 non-null    float64
 4   mode              833 non-null    int64  
 5   speechiness       833 non-null    float64
 6   acousticness      833 non-null    float64
 7   instrumentalness  833 non-null    float64
 8   liveness          833 non-null    float64
 9   valence           833 non-null    float64
 10  tempo             833 non-null    float64
 11  type              833 non-null    object 
 12  id                833 non-null    object 
 13  uri               833 non-null    object 
 14  track_href        833 non-null    object 
 15  analysis_url      833 non-null    object 
 16  duration_ms       833 non-null    int64  
 1

In [ ]:
# menghapus kolom yang tidak digunakan
columns_to_drop = ['analysis_url', 'track_href', 'type', 'uri']

japan_aufeat.drop(columns_to_drop, axis=1, inplace=True)
# mengganti kolom id jadi id track
japan_aufeat.rename(columns={'id': 'track id'}, inplace=True)
japan_aufeat.shape

(833, 14)

In [ ]:
# merge dataframe, antara df track dan fitur audio
# metode 'inner' memastikan kalau cuma keep track id yang sekarang di kedua dataset
japan_song_df = pd.merge(japan_track, japan_aufeat, on='track id', how='inner')
japan_song_df.head()

,artist name,track name,track id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Official HIGE DANdism,ミックスナッツ,60ReNXcPcsVFd7nUuQw5HZ,77,0.542,0.918,6,-5.074,1,0.0937,0.01010,0.000000,0.259,0.784,150.094,213438,4
1,Ish Kevin,My Year (2022),16TeLUubmneRAhwmRAMbjM,21,0.521,0.683,5,-6.240,0,0.5010,0.15900,0.000000,0.187,0.737,139.439,161808,3
2,Kenshi Yonezu,POP SONG,2wPSiRWMFFeL1FC4MJ3Qxf,50,0.762,0.792,11,-6.443,0,0.1340,0.22000,0.000017,0.939,0.806,109.968,199187,4
3,THE YELLOW MONKEY,JAM - 2022 Remaster,4uQkimBWBbDdDZdPo6GJsZ,46,0.282,0.862,0,-3.578,1,0.0484,0.00392,0.000000,0.107,0.366,113.808,319987,3
4,SEKAI NO OWARI,Habit,2uMNMcjjUz8oNIxjIu20qE,78,0.749,0.850,10,-5.390,0,0.1260,0.20300,0.000000,0.180,0.912,117.968,253154,4


In [ ]:
# liat ukuran jumlah baris dan kolom dataframe audio features
japan_aufeat.shape

(833, 14)

In [ ]:
# liat info dataframe gabungan 
japan_song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 833 entries, 0 to 832
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist name       833 non-null    object 
 1   track name        833 non-null    object 
 2   track id          833 non-null    object 
 3   popularity        833 non-null    int64  
 4   danceability      833 non-null    float64
 5   energy            833 non-null    float64
 6   key               833 non-null    int64  
 7   loudness          833 non-null    float64
 8   mode              833 non-null    int64  
 9   speechiness       833 non-null    float64
 10  acousticness      833 non-null    float64
 11  instrumentalness  833 non-null    float64
 12  liveness          833 non-null    float64
 13  valence           833 non-null    float64
 14  tempo             833 non-null    float64
 15  duration_ms       833 non-null    int64  
 16  time_signature    833 non-null    int64  
dt

In [ ]:
# periksa jumlah duplicate pada kolom artist name dan track name
japan_song_df[japan_song_df.duplicated(subset=['artist name', 'track name'], keep=False)].count()

artist name         0
track name          0
track id            0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64

## JKT48 Song Extraction

In [ ]:
# query buat pencarian
result = sp.search('JKT48')
# tampilkan dengan label track dan items
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2l8I5pWUnfF7bMK1z6EJRk'},
  'href': 'https://api.spotify.com/v1/artists/2l8I5pWUnfF7bMK1z6EJRk',
  'id': '2l8I5pWUnfF7bMK1z6EJRk',
  'name': 'JKT48',
  'type': 'artist',
  'uri': 'spotify:artist:2l8I5pWUnfF7bMK1z6EJRk'}]

ekstrak uri artisnya

kemudian tarik data pada album artisnya

In [ ]:
# extract  uri pada artisnya
artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
# tarik semua album pada artisnya 
sp_albums = sp.artist_albums(artist_uri)

setelah di extract albumnya

taro masing - masing nama ama uri nya ke list kosong, karena mengambil daftar lagu dengan perulangan

In [ ]:
# buat list kosong buat nampung nama album artis sama uri nya
album_names = []
album_uris = []

# tarik lagu dengan perulangan, dimana i mengakses setiap item lagu pada album
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
    
album_names
album_uris

['spotify:album:4daUqzU4eqX8JYcpZRnXx2',
 'spotify:album:6KR4SKOFo7RAO60pklOrBO',
 'spotify:album:3V92iPlKnzSclyl6E0IVn4',
 'spotify:album:1TW3TkuVNPV2N1BUFDKYF0',
 'spotify:album:0Rhk7pQXZsalweggVY6p8o',
 'spotify:album:4nF5OeX7uE5yOs5Zi3eP3r',
 'spotify:album:6VM2ed8eCG2yrpUS4R9sCj',
 'spotify:album:7JR86QGDS5O0u185jepqaX',
 'spotify:album:7xa2B49fYCaKigtBI1zkW7',
 'spotify:album:6r0wtZqJ6giE5JieQTwPse',
 'spotify:album:3Q8kUwEAmYC8Ywublc14qn',
 'spotify:album:6h9KWIpNduJQWcwe86YMEU',
 'spotify:album:2ra7LJhwBLf0A7LmZR8WkI',
 'spotify:album:652jsJs6hnACmycOLVL9wm',
 'spotify:album:47tiUmCalb8rA6PDq8UJ88',
 'spotify:album:0FiXJC94RTKMPRxJdZamC0',
 'spotify:album:5ow7HbvEbScZ2F3xjGnlU3',
 'spotify:album:5Jn3DUq1wJzaaiOp2VOiMA',
 'spotify:album:6BxN7xumuu2ivJlA4TnnsK',
 'spotify:album:4GTnA1wHswtH4XJliZOEgL']

### Ambil Data lagu dari Tiap Album
menggunakan looping dengan index yang mengakes tiap album buat mengekstrak key pada data track albumnya

In [ ]:
# diakses dengan uri
def albumSongs(uri):
    # assign album uri ke nama album
    album = uri 
    # buat dictionary koosong buat nentuin albumnya 
    # buat key value biar dimasukin ke dictionary albumnya
    spotify_albums[album] = {} 
    # bikin list kosong
    spotify_albums[album]['album'] = []
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    # narik data dari track album
    tracks = sp.album_tracks(album) 
    # pengulangan buat tiap lagunya 
    for n in range(len(tracks['items'])): 
        # nambahin nama album yang di track pake album_count
        spotify_albums[album]['album'].append(album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

trus jalanin function ke tiap album uri ke list buat narik datanya

trus ditampung ke dict kosong buat nampung data albumnya

In [ ]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    albumSongs(i)
    print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
    album_count+=1 #Updates album count once all tracks have been added

Album Sambil Menggandeng Erat Tanganku songs has been added to spotify_albums dictionary
Album Tadaima Renaichuu songs has been added to spotify_albums dictionary
Album Theater no Megami: Dewi Theater songs has been added to spotify_albums dictionary
Album Bel Terakhir Berbunyi (Theater Live Version) songs has been added to spotify_albums dictionary
Album Seishun Girls - Gadis Gadis Remaja songs has been added to spotify_albums dictionary
Album Dareka no Tame ni - Demi Seseorang songs has been added to spotify_albums dictionary
Album Pajama Drive songs has been added to spotify_albums dictionary
Album Joy Kick! Tears songs has been added to spotify_albums dictionary
Album B.E.L.I.E.V.E songs has been added to spotify_albums dictionary
Album Mahagita: Kamikyokutachi songs has been added to spotify_albums dictionary
Album Boku No Taiyou: Matahari Milikku songs has been added to spotify_albums dictionary
Album Renai Kinshi Jourei: Aturan Anti Cinta songs has been added to spotify_albums d

### Grab fitur audio buat tiap lagu
bkin key values buat nampung fitur atau variabel kolom pada tiap track album

trus tambahin ke list yang merupakan representasi semua track musik pada album

In [ ]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['key'] = []
    spotify_albums[album]['mode'] = []
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['duration_ms'] = []
    spotify_albums[album]['time_signature'] = []
    spotify_albums[album]['popularity'] = []

    # bikin track counter buat variabel inisiasi pada perulangan
    track_count = 0
    for track in spotify_albums[album]['uri']:
        # tarik fitur audio per track
        print("Requesting " + track)
        features = sp.audio_features(track)
        
        # masukin ke key value yang sesuai
        spotify_albums[album]['key'].append(features[0]['key'])
        spotify_albums[album]['mode'].append(features[0]['mode'])
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['duration_ms'].append(features[0]['duration_ms'])
        spotify_albums[album]['time_signature'].append(features[0]['time_signature'])
        # popularitas juga ditampung
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

looping pada album untuk mengekstrak fitur audio nya

In [ ]:
request_count = 0
for i in spotify_albums:
    audio_features(i)
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " playlists completed")
        print('Loop #: {}'.format(request_count))

Requesting spotify:track:7mGO7lGxnJhMWtqERZig8l
Requesting spotify:track:1TdhewZV5VYPZY1ahpfgBE
Requesting spotify:track:4fhJ1jFoIapbdiW8aDccHc
Requesting spotify:track:0eotVbveGVkC4gPvzdvks5
Requesting spotify:track:2N6KAievdXztAwHzcymJdz
Requesting spotify:track:1DUrPNEqWWVLYZgBnikiEQ
Requesting spotify:track:1LRNtjqpe2KVU53TnKkZLm
Requesting spotify:track:0pI9PVL8ROOIumjjLczqiZ
Requesting spotify:track:6Idrb4GBzCUGaP7G40G5XT
Requesting spotify:track:23r4tuEuW62Pv8qT40cstv
Requesting spotify:track:4G22d4UONZNFc1Dk4GPvc6
Requesting spotify:track:5fITh6M3sCMDGYjXmAWWQr
Requesting spotify:track:6JUEOP9V9N6qvGweYVR618
Requesting spotify:track:0Eqh5OgF8zDrFTP3TH1eWz
Requesting spotify:track:2FRQwszGqh5bVcMStIM7iA
Requesting spotify:track:4BhaqBmF4CylMsdP0ARzRO
Requesting spotify:track:2ZZb1KxDrgG8Eudlz9tvcW
Requesting spotify:track:58Q6v2XVCukU3RovLLTpFZ
Requesting spotify:track:55ru893ySTy7Ot2uJoD9ZR
Requesting spotify:track:1lDNAHck7TAhHCXtghpyDK
Requesting spotify:track:1ZcKRoOvykD6Fwi

### masukin data ke tabel dataframe

In [ ]:
dic_jkt = {}
dic_jkt['album'] = []
dic_jkt['track_number'] = []
dic_jkt['id'] = []
dic_jkt['name'] = []
dic_jkt['uri'] = []
dic_jkt['key'] = []
dic_jkt['mode'] = []
dic_jkt['danceability'] = []
dic_jkt['acousticness'] = []
dic_jkt['energy'] = []
dic_jkt['liveness'] = []
dic_jkt['loudness'] = []
dic_jkt['tempo'] = []
dic_jkt['valence'] = []
dic_jkt['instrumentalness'] = []
dic_jkt['speechiness'] = []
dic_jkt['duration_ms'] = []
dic_jkt['time_signature'] = []
dic_jkt['popularity'] = []
for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_jkt[feature].extend(spotify_albums[album][feature])
        
len(dic_jkt['album'])

203

masukin ke dataframe

In [ ]:
df_jkt = pd.DataFrame.from_dict(dic_jkt)
df_jkt.head(20)

,album,track_number,id,name,uri,key,mode,danceability,acousticness,energy,liveness,loudness,tempo,valence,instrumentalness,speechiness,duration_ms,time_signature,popularity
0,Sambil Menggandeng Erat Tanganku,1,7mGO7lGxnJhMWtqERZig8l,Angin Kita,spotify:track:7mGO7lGxnJhMWtqERZig8l,5,1,0.554,0.05820,0.583,0.3260,-11.260,106.997,0.662,0.000000,0.0343,279708,4,27
1,Sambil Menggandeng Erat Tanganku,2,1TdhewZV5VYPZY1ahpfgBE,Mango No. 2,spotify:track:1TdhewZV5VYPZY1ahpfgBE,1,0,0.614,0.21200,0.602,0.0829,-11.706,170.008,0.795,0.000000,0.0488,231404,4,25
2,Sambil Menggandeng Erat Tanganku,3,4fhJ1jFoIapbdiW8aDccHc,Sambil Menggandeng Erat Tanganku,spotify:track:4fhJ1jFoIapbdiW8aDccHc,5,1,0.515,0.03440,0.692,0.8020,-10.994,163.954,0.642,0.000000,0.0520,264684,4,32
3,Sambil Menggandeng Erat Tanganku,4,0eotVbveGVkC4gPvzdvks5,Bel Sekolah adalah Love Song,spotify:track:0eotVbveGVkC4gPvzdvks5,7,1,0.452,0.01670,0.625,0.3350,-11.097,175.068,0.679,0.000000,0.0429,232121,4,31
4,Sambil Menggandeng Erat Tanganku,5,2N6KAievdXztAwHzcymJdz,Glory Days,spotify:track:2N6KAievdXztAwHzcymJdz,0,1,0.553,0.08230,0.752,0.0674,-12.588,172.056,0.816,0.000000,0.0340,295535,4,28
5,Sambil Menggandeng Erat Tanganku,6,1DUrPNEqWWVLYZgBnikiEQ,Barcode,spotify:track:1DUrPNEqWWVLYZgBnikiEQ,3,0,0.626,0.03230,0.592,0.3700,-11.624,135.984,0.789,0.000378,0.0285,280783,4,22
6,Sambil Menggandeng Erat Tanganku,7,1LRNtjqpe2KVU53TnKkZLm,Ajak Aku Pergi Menuju ke Wimbledo,spotify:track:1LRNtjqpe2KVU53TnKkZLm,7,1,0.611,0.09930,0.689,0.3460,-11.203,147.070,0.878,0.000000,0.0315,251513,4,26
7,Sambil Menggandeng Erat Tanganku,8,0pI9PVL8ROOIumjjLczqiZ,Sang Pianis Hujan,spotify:track:0pI9PVL8ROOIumjjLczqiZ,6,0,0.552,0.00433,0.733,0.3440,-12.251,145.026,0.805,0.000000,0.0302,277500,4,25
8,Sambil Menggandeng Erat Tanganku,9,6Idrb4GBzCUGaP7G40G5XT,Keberadaan Coklat Itu,spotify:track:6Idrb4GBzCUGaP7G40G5XT,0,1,0.648,0.01650,0.842,0.3260,-11.833,125.077,0.744,0.000824,0.0537,286434,4,21
9,Sambil Menggandeng Erat Tanganku,10,23r4tuEuW62Pv8qT40cstv,Innocence,spotify:track:23r4tuEuW62Pv8qT40cstv,1,0,0.638,0.00978,0.736,0.3530,-11.822,132.008,0.738,0.000044,0.0493,251274,4,24


buat dataframe finalnya

dengan mengurutkan berdasarkan popularitas dan menghapus duplikasinya

In [ ]:
lagu_jkt48 = df_jkt.sort_values('popularity', ascending=False).drop_duplicates('name').sort_index()
lagu_jkt48.shape

(195, 19)

## Save all Dataframe to CSV

In [ ]:
# kalau mau di simpan ke dalam csv
indo_song_df.to_csv('spotify_song_indonesia.csv', index=False)
korean_song_df.to_csv('spotify_song_korea.csv', index=False)
japan_song_df.to_csv('spotify_song_japan.csv', index=False)
lagu_jkt48.to_csv('lagu_jkt48.csv', index=False)